# Revised and simplified version of cs2

In [1]:
#what else can be done or any question
#normalize the x? before training
#how to undo normalized in normalization
#

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Simple Exploratory Data Analysis

In [ ]:
df = pd.read_csv("heart.csv")

In [ ]:
df.head()

In [ ]:
#change name column to lowercase is easier to proceed

# Make column labels lowercase
df.columns = map(str.lower, df.columns)

# Replace spaces in columns with underscore
df.columns = df.columns.str.replace('\s+', '_')

In [ ]:
#number of rows and columns
print('Data: \n')
print("Number of columns: "+ str(df.shape[1]))
print("Number of rows: "+ str(df.shape[0]))

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(df.corr(),annot=True,cmap='YlGnBu',fmt='.2f',linewidths=2)

In [ ]:
df['target'].value_counts()

In [ ]:
sns.distplot(df['age'],color='Red',hist_kws={'alpha':1,"linewidth": 2}, kde_kws={"color": "k", "lw": 3, "label": "KDE"})


# Preprocessing 

In [ ]:
#check outlier
#1.iqr
#2.zscore

In [ ]:
#using zscore to detect outliers
#good god im lazy...im using zscore because its looks easier to do
df.shape

In [ ]:
z = np.abs(stats.zscore(df))

In [ ]:
z = np.abs(stats.zscore(df))
z

In [ ]:
df1 = df[(z < 3).all(axis=1)]

In [ ]:
df1.shape

In [ ]:
#check missing data

In [ ]:
df1.isnull().sum()

In [ ]:
df1.dtypes

In [ ]:
df1.loc[df['thal']==0]

df1['thal'] = df['thal'].replace(0,2)

df1['thal'].unique()

In [ ]:
#feature engineering

In [ ]:
#some of the features are categorical...
#sex,cp,fbs,restecg,exang,slope,ca,thal,target
#double check this with the definition documents
df.head()

In [ ]:
df1['sex']=df1['sex'].astype('category')
df1['cp']=df1['cp'].astype('category')
df1['fbs']=df1['fbs'].astype('category')
df1['restecg']=df1['restecg'].astype('category')
df1['exang']=df1['exang'].astype('category')
df1['slope']=df1['slope'].astype('category')
df1['ca']=df1['ca'].astype('category')
df1['thal']=df1['thal'].astype('category')
df1['target']=df1['target'].astype('category')

In [ ]:
df1.dtypes

#one hot encoding 
#use this for those categorical datatype..rmmbr to remove first dupe

df1=pd.get_dummies(df1,drop_first=True)
df1.head()

# machine learning 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,plot_confusion_matrix
from joblib import dump
from sklearn import metrics

In [ ]:
x = df1.drop('target',axis=1)
y = df1['target']

In [ ]:
x_train,x_test, y_train, y_test = train_test_split(x,y, test_size = 0.1,random_state=101)

In [ ]:
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
log_model= LogisticRegression(solver='saga',multi_class="ovr",max_iter=5000)

In [ ]:
#penalty
penalty= ['l1']

#use logarithmic spaced c values
C = [1.0]

In [ ]:
grid_model=GridSearchCV(log_model,param_grid={'C':C,'penalty':penalty})

In [ ]:
grid_model.fit(x_train,y_train)

In [ ]:
best_model = grid_model.best_estimator_

In [ ]:
a =best_model.coef_
a

In [ ]:
y_pred =grid_model.predict(x_test)

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:

cm = metrics.confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
#save the scaler and model for deployment
dump(scaler,"casestudy2.pkl")

In [ ]:
dump(grid_model,"casestudy2.h5")

In [ ]:
from joblib import dump, load

heart_model = load("casestudy2.h5")

heart_scaler = load("casestudy2.pkl")

def return_prediction(model,scaler,sample_json):
    
    age = sample_json['age']
    sex = sample_json['sex']
    cp = sample_json['cp']
    trestbps = sample_json['trestbps']
    chol = sample_json['chol']
    fbs = sample_json['fbs']
    restecg = sample_json['restecg']
    thalach = sample_json['thalach']
    exang = sample_json['exang']
    oldpeak = sample_json['oldpeak']
    slope = sample_json['slope']
    ca = sample_json['ca']
    thal = sample_json['thal']
    
    heart = [[age,sex,cp,trestbps,chol,fbs,restecg,thalach,
       exang,oldpeak,slope,ca,thal]]
    
    heart = scaler.transform(heart)
    
    prediction= model.predict(heart)

    return prediction[0]

In [ ]:
heart_pred = {'age':63,
              'sex':1,
              'cp':3,
              'trestbps':145,
              'chol':233,
              'fbs':1,
              'restecg':0,
              'thalach':150,
              'exang':0,
              'oldpeak':2.3,
              'slope':0,
              'ca':0,
              'thal':1}             

In [ ]:
prediction = return_prediction(heart_model,heart_scaler,heart_pred)

In [ ]:
prediction